In [1]:
### インポート ###
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re
import csv
import pandas as pd
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, TypeVar, Generic
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# =====
# 設定
# =====

### 設定ファイル ###
df_config_racecourse = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="racecourse", header=0)
df_config_style = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="style", header=0)
df_spat4 = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="spat4", header=0)
df_purchase = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="purchase", header=0)
df_config_scrape = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="scrape", header=None, index_col=0)
df_config_netkeiba = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="netkeiba", header=None, index_col=0)
df_config_gmail = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="gmail", header=0)
# 対象競馬場とレース
PLACE_MAP = df_config_racecourse.set_index('key')['value'].to_dict()
print(f"競馬場: {df_config_gmail}")
# 脚質
STYLE_MAP = df_config_style.set_index('key')['value'].to_dict()
print(f"脚質: {STYLE_MAP}")
# spat4
SPAT4_MAP = df_spat4.set_index('key')['value'].to_dict()
MODE = SPAT4_MAP.get('MODE')
PRE_MINUTES = SPAT4_MAP.get('PRE_MINUTES')
LOGIN_URL_SPAT4 = SPAT4_MAP.get('LOGIN_URL')
PRE_MINUTES = SPAT4_MAP.get('PRE_MINUTES')
SUBSCRIBER_ID = SPAT4_MAP.get('加入者番号')
USER_ID = SPAT4_MAP.get('利用者ID')
PASSCODE = SPAT4_MAP.get('暗証番号')
URL_ODDS_BET = SPAT4_MAP.get('URL_オッズ投票')
print(f'MODE: {MODE}モード')
print(f'購入前時間: {PRE_MINUTES}分')
print(f'LOGIN_URL: {LOGIN_URL_SPAT4}')
print(f'加入者番号: {SUBSCRIBER_ID}')
print(f'利用者ID: {USER_ID}')
print(f'暗証番号: {PASSCODE}')
print(f'オッズ投票画面のURL: {URL_ODDS_BET}')
# 期待値の閾値
PURCHASE_MAP = df_purchase.set_index('key')['value'].to_dict()
print(f'期待値の閾値: {PURCHASE_MAP}')
# gmail
GMAIL_MAP = df_config_gmail.set_index('key')['value'].to_dict()
print(f'gmail設定: {GMAIL_MAP}')
# スクレイピング
PATH_CHROME_DRIVER = df_config_scrape.loc["PATH_CHROME_DRIVER"].iloc[0]
# netkeiba
LOGIN_URL = df_config_netkeiba.loc["LOGIN_URL"].iloc[0]
LOGIN_ID = df_config_netkeiba.loc["LOGIN_ID"].iloc[0]
LOGIN_PASSWORD = df_config_netkeiba.loc["LOGIN_PASSWORD"].iloc[0]
RACE_LIST_URL = df_config_netkeiba.loc["RACE_LIST_URL"].iloc[0]
MODE = df_config_netkeiba.loc["MODE"].iloc[0]
# レース単位
print(f"MODE: {MODE}")
if MODE == "shutuba":
    SELECTOR_RACECOURSE = df_config_netkeiba.loc["SELECTOR_RACECOURSE_SHUTUBA"].iloc[0]
    SELECTOR_RACE_NUMBER = df_config_netkeiba.loc["SELECTOR_RACE_NUMBER_SHUTUBA"].iloc[0]
    SELECTOR_NUM_HORSES = df_config_netkeiba.loc["SELECTOR_NUM_HORSES_SHUTUBA"].iloc[0]
    XPATH_RACE_INFO = df_config_netkeiba.loc["XPATH_RACE_INFO_SHUTUBA"].iloc[0]
elif MODE == "result":
    SELECTOR_RACECOURSE = df_config_netkeiba.loc["SELECTOR_RACECOURSE_RESULT"].iloc[0]
    SELECTOR_RACE_NUMBER = df_config_netkeiba.loc["SELECTOR_RACE_NUMBER_RESULT"].iloc[0]
    SELECTOR_NUM_HORSES = df_config_netkeiba.loc["SELECTOR_NUM_HORSES_RESULT"].iloc[0]
    XPATH_RACE_INFO = df_config_netkeiba.loc["XPATH_RACE_INFO_RESULT"].iloc[0]
XPATH_RELIABILITY = df_config_netkeiba.loc["XPATH_RELIABILITY"].iloc[0]
XPATH_OPINION = df_config_netkeiba.loc["XPATH_OPINION"].iloc[0]
XPATH_CORNER_1 = df_config_netkeiba.loc["XPATH_CORNER_1"].iloc[0]
XPATH_CORNER_2 = df_config_netkeiba.loc["XPATH_CORNER_2"].iloc[0]
XPATH_CORNER_3 = df_config_netkeiba.loc["XPATH_CORNER_3"].iloc[0]
XPATH_RANK_1_CORNER = df_config_netkeiba.loc["XPATH_RANK_1_CORNER"].iloc[0]
XPATH_RANK_2_CORNER = df_config_netkeiba.loc["XPATH_RANK_2_CORNER"].iloc[0]
XPATH_RANK_3_CORNER = df_config_netkeiba.loc["XPATH_RANK_3_CORNER"].iloc[0]
XPATH_RANK_4_CORNER = df_config_netkeiba.loc["XPATH_RANK_4_CORNER"].iloc[0]
# 馬単位（shutuba）
XPATH_HORSE_NUMBER_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_1_SHUTUBA"].iloc[0]
XPATH_HORSE_NUMBER_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_2_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_1_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_1_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_1_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_2_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_2_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_1_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_2_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_2_SHUTUBA"].iloc[0]
XPATH_JOCKEY_LINK_1_SHUTUBA = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_1_SHUTUBA"].iloc[0]
XPATH_JOCKEY_LINK_2_SHUTUBA = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_2_SHUTUBA"].iloc[0]
XPATH_POPULARITY_1_SHUTUBA = df_config_netkeiba.loc["XPATH_POPULARITY_1_SHUTUBA"].iloc[0]
XPATH_POPULARITY_2_SHUTUBA = df_config_netkeiba.loc["XPATH_POPULARITY_2_SHUTUBA"].iloc[0]
XPATH_ODDS_1_SHUTUBA = df_config_netkeiba.loc["XPATH_ODDS_1_SHUTUBA"].iloc[0]
XPATH_ODDS_2_SHUTUBA = df_config_netkeiba.loc["XPATH_ODDS_2_SHUTUBA"].iloc[0]
XPATH_IMPOSE_1_SHUTUBA = df_config_netkeiba.loc["XPATH_IMPOSE_1_SHUTUBA"].iloc[0]
XPATH_IMPOSE_2_SHUTUBA = df_config_netkeiba.loc["XPATH_IMPOSE_2_SHUTUBA"].iloc[0]
XPATH_FRAME_1_SHUTUBA = df_config_netkeiba.loc["XPATH_FRAME_1_SHUTUBA"].iloc[0]
XPATH_FRAME_2_SHUTUBA = df_config_netkeiba.loc["XPATH_FRAME_2_SHUTUBA"].iloc[0]
XPATH_WEIGHT_1_SHUTUBA = df_config_netkeiba.loc["XPATH_WEIGHT_1_SHUTUBA"].iloc[0]
XPATH_WEIGHT_2_SHUTUBA = df_config_netkeiba.loc["XPATH_WEIGHT_2_SHUTUBA"].iloc[0]
XPATH_SEX_AND_AGE_1_SHUTUBA = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_1_SHUTUBA"].iloc[0]
XPATH_SEX_AND_AGE_2_SHUTUBA = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_2_SHUTUBA"].iloc[0]
XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA"].iloc[0]
XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA"].iloc[0]
# 馬単位（result）
XPATH_FINISH_RANK_1_RESULT = df_config_netkeiba.loc["XPATH_FINISH_RANK_1_RESULT"].iloc[0]
XPATH_FINISH_RANK_2_RESULT = df_config_netkeiba.loc["XPATH_FINISH_RANK_2_RESULT"].iloc[0]
XPATH_FRAME_NUMBER_1_RESULT = df_config_netkeiba.loc["XPATH_FRAME_NUMBER_1_RESULT"].iloc[0]
XPATH_FRAME_NUMBER_2_RESULT = df_config_netkeiba.loc["XPATH_FRAME_NUMBER_2_RESULT"].iloc[0]
XPATH_HORSE_NUMBER_1_RESULT = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_1_RESULT"].iloc[0]
XPATH_HORSE_NUMBER_2_RESULT = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_2_RESULT"].iloc[0]
XPATH_HORSE_LINK_1_RESULT = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_RESULT"].iloc[0]
XPATH_HORSE_LINK_2_RESULT = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_RESULT"].iloc[0]
XPATH_SEX_AND_AGE_1_RESULT = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_1_RESULT"].iloc[0]
XPATH_SEX_AND_AGE_2_RESULT = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_2_RESULT"].iloc[0]
XPATH_IMPOSE_1_RESULT = df_config_netkeiba.loc["XPATH_IMPOSE_1_RESULT"].iloc[0]
XPATH_IMPOSE_2_RESULT = df_config_netkeiba.loc["XPATH_IMPOSE_2_RESULT"].iloc[0]
XPATH_JOCKEY_LINK_1_RESULT = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_1_RESULT"].iloc[0]
XPATH_JOCKEY_LINK_2_RESULT = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_2_RESULT"].iloc[0]
XPATH_TIME_1_RESULT = df_config_netkeiba.loc["XPATH_TIME_1_RESULT"].iloc[0]
XPATH_TIME_2_RESULT = df_config_netkeiba.loc["XPATH_TIME_2_RESULT"].iloc[0]
XPATH_DIFF_1_RESULT = df_config_netkeiba.loc["XPATH_DIFF_1_RESULT"].iloc[0]
XPATH_DIFF_2_RESULT = df_config_netkeiba.loc["XPATH_DIFF_2_RESULT"].iloc[0]
XPATH_POPULARITY_1_RESULT = df_config_netkeiba.loc["XPATH_POPULARITY_1_RESULT"].iloc[0]
XPATH_POPULARITY_2_RESULT = df_config_netkeiba.loc["XPATH_POPULARITY_2_RESULT"].iloc[0]
XPATH_ODDS_1_RESULT = df_config_netkeiba.loc["XPATH_ODDS_1_RESULT"].iloc[0]
XPATH_ODDS_2_RESULT = df_config_netkeiba.loc["XPATH_ODDS_2_RESULT"].iloc[0]
XPATH_LAST_3_FURLONGS_1_RESULT = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_1_RESULT"].iloc[0]
XPATH_LAST_3_FURLONGS_2_RESULT = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_2_RESULT"].iloc[0]
XPATH_WEIGHT_1_RESULT = df_config_netkeiba.loc["XPATH_WEIGHT_1_RESULT"].iloc[0]
XPATH_WEIGHT_2_RESULT = df_config_netkeiba.loc["XPATH_WEIGHT_2_RESULT"].iloc[0]

競馬場:             key                  value  備考
0          from  keibaaitool@gmail.com NaN
1            to  keibaaitool@gmail.com NaN
2  app_password    nkrd rzhz bske ktyj NaN
脚質: {1: '逃げ', 2: '先行', 3: '差し', 4: '追込'}
MODE: testモード
購入前時間: 37分
LOGIN_URL: https://www.spat4.jp/keiba/pc
加入者番号: 12118802
利用者ID: 19930403
暗証番号: 1979
オッズ投票画面のURL: https://www1.spat4.jp/keiba/pc?HANDLERR=P120S&RACER=1
期待値の閾値: {'th_num8_pop1': 1, 'th_num8_pop2': 1, 'th_num8_pop3': 1, 'th_num8_pop4': 1, 'th_num8_pop5': 1, 'th_num8_pop6': 1, 'th_num8_pop7': 1, 'th_num8_pop8': 1, 'th_num9_pop1': 1, 'th_num9_pop2': 1, 'th_num9_pop3': 1, 'th_num9_pop4': 1, 'th_num9_pop5': 1, 'th_num9_pop6': 1, 'th_num9_pop7': 1, 'th_num9_pop8': 1, 'th_num9_pop9': 1, 'th_num10_pop1': 1, 'th_num10_pop2': 1, 'th_num10_pop3': 1, 'th_num10_pop4': 1, 'th_num10_pop5': 1, 'th_num10_pop6': 1, 'th_num10_pop7': 1, 'th_num10_pop8': 1, 'th_num10_pop9': 1, 'th_num10_pop10': 1, 'th_num11_pop1': 1, 'th_num11_pop2': 1, 'th_num11_pop3': 1, 'th_num11_pop

In [3]:
### クラス ###

# 馬データ（Shutuba）
@dataclass
class Horse_Shutuba:
    """出走馬のデータを保持するデータクラス"""
    horse_id: str
    horse_name: str
    jockey_id: str
    jockey_name: str
    popularity: str
    odds: str
    sex_and_age: str
    weight: str
    horse_number: str
    frame_number: str
    position_1_top_pred: str
    position_1_left_pred: str
    position_2_top_pred: str
    position_2_left_pred: str
    position_3_top_pred: str
    position_3_left_pred: str
    position_4_top_pred: str
    position_4_left_pred: str 
    position_1_top_pred_jockey_tendency: str
    position_1_left_pred_jockey_tendency: str
    position_2_top_pred_jockey_tendency: str
    position_2_left_pred_jockey_tendency: str
    position_3_top_pred_jockey_tendency: str
    position_3_left_pred_jockey_tendency: str
    position_4_top_pred_jockey_tendency: str
    position_4_left_pred_jockey_tendency: str
    style_pred: str
    impost: str
    last_3_furlongs_pred: str

# 馬データ（Result）
@dataclass
class Horse_Result:
    """出走馬のデータを保持するデータクラス"""
    finish_rank: str
    frame_number: str
    horse_number: str
    horse_id: str
    horse_name: str
    sex_and_age: str
    impost: str    
    jockey_id: str
    jockey_name: str
    time: str
    diff: str    
    popularity: str
    odds: str
    last_3_furlongs: str
    weight: str

# レースデータ（Shutuba）
T = TypeVar('T')
@dataclass
class Race:
    """レースの基本情報と出走馬リストを保持するデータクラス"""
    race_id: str
#    race_name: str
    race_date: str
    race_time: str
    num_horses: str
    race_number: int
#    weather_name: str
#    track_condition_name: str
    racecourse: str
    ground: str
    distance: str
    direction:  str
    reliability: str
    opinion: str
    rank_1_corner: str
    rank_2_corner: str
    rank_3_corner: str
    rank_4_corner: str    
    horses: List[T] = field(default_factory=list)

In [5]:
# ========
# メソッド
# ========

def is_xpath_present(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath)) > 0

def extract_style_values(style_str):
    """
    style文字列からtopとleftの数値を抽出する関数（マイナス対応版）
    """
    # 正規表現の解説:
    # -?    : マイナス記号があってもなくても良い
    # [\d\.]+ : 数字またはドットが1回以上続く
    # ()    : このカッコ内の部分を抽出する
    top_match = re.search(r"top:\s*(-?[\d\.]+)%", style_str)
    left_match = re.search(r"left:\s*(-?[\d\.]+)%", style_str)
    
    # 抽出した文字列をfloatに変換
    top_val = float(top_match.group(1)) if top_match else None
    left_val = float(left_match.group(1)) if left_match else None
    
    return top_val, left_val

def get_position_pred(driver):
    # HorseIconクラスを持つすべての要素を取得（リスト形式で返ってくる）
    position_pred_elements = driver.find_elements(By.CLASS_NAME, "HorseIcon")
    # 結果を格納する辞書
    dic_position_pred = {}
    for element in position_pred_elements:
        horse_number = element.get_attribute("id")
        position_pred = element.get_attribute("style")
        top_val, left_val = extract_style_values(position_pred)
        dic_val = {}
        dic_val["top"] = top_val
        dic_val["left"] = left_val
        # idが空でない場合のみ追加
        if horse_number:
            dic_position_pred[horse_number] = dic_val
    # 戻り値
    return dic_position_pred

def login_netkeiba(driver):
    # ログイン
    driver.get(LOGIN_URL)
    driver.find_element(By.NAME, "login_id").send_keys(LOGIN_ID)
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys(LOGIN_PASSWORD)
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

def login_spat4(driver):
    # ログイン
    driver.get(SPAT4_MAP.get('LOGIN_URL'))
    driver.find_element(By.XPATH, "//*[@id='MEMBERNUMR']").send_keys(SPAT4_MAP.get('加入者番号'))
    driver.find_element(By.XPATH, "//*[@id='MEMBERIDR']").send_keys(SPAT4_MAP.get('利用者ID'))
    driver.find_element(By.XPATH, "/html/body/div/div[1]/form/a/span/div").click()
    time.sleep(3)

def scrape_horse_shutuba_data(driver, num_horses):
    
    all_horses_data = []
    # 馬番・馬名取得
    # 馬番・馬名・馬名リンク取得
    for i in range(1, num_horses + 1):
        try:
            # 馬番
            xpath_horse_number = XPATH_HORSE_NUMBER_1_SHUTUBA + str(i) + XPATH_HORSE_NUMBER_2_SHUTUBA
            if is_xpath_present(driver, xpath_horse_number): # 通常通り出走
                horse_number = driver.find_element(By.XPATH, xpath_horse_number).text.strip()
            elif is_xpath_present(driver, f"//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[3]/table/tbody/tr[{str(i)}]/td[2]"): # 取消等
                horse_number = driver.find_element(By.XPATH, f"//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[3]/table/tbody/tr[{str(i)}]/td[2]").text.strip()
                horse_data = Horse_Shutuba(
                    horse_id = None,
                    horse_name = None,
                    jockey_id = None,
                    jockey_name = None,
                    popularity = None,
                    odds = None,
                    sex_and_age = None,
                    weight = None,
                    horse_number = horse_number,
                    frame_number = None,
                    position_1_top_pred = None,
                    position_1_left_pred = None,
                    position_2_top_pred = None,
                    position_2_left_pred = None,
                    position_3_top_pred = None,
                    position_3_left_pred = None,
                    position_4_top_pred = None,
                    position_4_left_pred = None,
                    position_1_top_pred_jockey_tendency = None,
                    position_1_left_pred_jockey_tendency = None,
                    position_2_top_pred_jockey_tendency = None,
                    position_2_left_pred_jockey_tendency = None,
                    position_3_top_pred_jockey_tendency = None,
                    position_3_left_pred_jockey_tendency = None,
                    position_4_top_pred_jockey_tendency = None,
                    position_4_left_pred_jockey_tendency = None,
                    style_pred = None,
                    impost = None,
                    last_3_furlongs_pred = None
                )
                all_horses_data.append(horse_data)
                continue
            else:
                break
            # 馬名と馬ID
            xpath_horse_link_1 = XPATH_HORSE_LINK_1_1_SHUTUBA + str(i) + XPATH_HORSE_LINK_1_2_SHUTUBA
            xpath_horse_link_2 = XPATH_HORSE_LINK_2_1_SHUTUBA + str(i) + XPATH_HORSE_LINK_2_2_SHUTUBA
            if is_xpath_present(driver, xpath_horse_link_1):
                xpath_horse_link = xpath_horse_link_1
            elif is_xpath_present(driver, xpath_horse_link_2):
                xpath_horse_link = xpath_horse_link_2
            horse_link_elem = driver.find_element(By.XPATH, xpath_horse_link)
            horse_name = horse_link_elem.text.strip()
            horse_href = horse_link_elem.get_attribute("href")
            horse_id = horse_href.rstrip("/").split("/")[-1]
            # 騎手名と騎手ID
            xpath_jockey_link = XPATH_JOCKEY_LINK_1_SHUTUBA + str(i) + XPATH_JOCKEY_LINK_2_SHUTUBA
            jockey_link_elem = driver.find_element(By.XPATH, xpath_jockey_link)
            jockey_name = jockey_link_elem.text.strip()
            jockey_href = jockey_link_elem.get_attribute("href")
            jockey_id = jockey_href.rstrip("/").split("/")[-1]
            # 人気
            xpath_popularity = XPATH_POPULARITY_1_SHUTUBA + str(i) + XPATH_POPULARITY_2_SHUTUBA
            popularity = driver.find_element(By.XPATH, xpath_popularity).text
            # オッズ
            xpath_odds = XPATH_ODDS_1_SHUTUBA + str(i) + XPATH_ODDS_2_SHUTUBA
            odds = driver.find_element(By.XPATH, xpath_odds).text
            # 性齢
            xpath_sex_and_age = XPATH_SEX_AND_AGE_1_SHUTUBA + str(i) + XPATH_SEX_AND_AGE_2_SHUTUBA
            sex_and_age = driver.find_element(By.XPATH, xpath_sex_and_age).text
            # 馬体重
            xpath_weight = XPATH_WEIGHT_1_SHUTUBA + str(i) + XPATH_WEIGHT_2_SHUTUBA
            weight = driver.find_element(By.XPATH, xpath_weight).text
            # 枠番
            xpath_frame_number = XPATH_FRAME_1_SHUTUBA + str(i) + XPATH_FRAME_2_SHUTUBA
            frame_number = driver.find_element(By.XPATH, xpath_frame_number).text
            # 斤量
            xpath_impost = XPATH_IMPOSE_1_SHUTUBA + str(i) + XPATH_IMPOSE_2_SHUTUBA
            impost = driver.find_element(By.XPATH, xpath_impost).text
            # 後半3F
            xpath_last_3_furlongs_pred = XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA + str(i) + XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA
            if is_xpath_present(driver, xpath_last_3_furlongs_pred):
                last_3_furlongs_pred = driver.find_element(By.XPATH, xpath_last_3_furlongs_pred).text
            else:
                last_3_furlongs_pred = None
            # 馬データ作成
            horse_data = Horse_Shutuba(
                horse_id = horse_id,
                horse_name = horse_name,
                jockey_id = jockey_id,
                jockey_name = jockey_name,
                popularity = popularity,
                odds = odds,
                sex_and_age = sex_and_age,
                weight = weight,
                horse_number = horse_number,
                frame_number = frame_number,
                position_1_top_pred = None,
                position_1_left_pred = None,
                position_2_top_pred = None,
                position_2_left_pred = None,
                position_3_top_pred = None,
                position_3_left_pred = None,
                position_4_top_pred = None,
                position_4_left_pred = None,
                position_1_top_pred_jockey_tendency = None,
                position_1_left_pred_jockey_tendency = None,
                position_2_top_pred_jockey_tendency = None,
                position_2_left_pred_jockey_tendency = None,
                position_3_top_pred_jockey_tendency = None,
                position_3_left_pred_jockey_tendency = None,
                position_4_top_pred_jockey_tendency = None,
                position_4_left_pred_jockey_tendency = None,
                style_pred = None,
                impost = impost,
                last_3_furlongs_pred = last_3_furlongs_pred
            )
            
            all_horses_data.append(horse_data)
        except Exception as e:
            print(f"  馬番{i}: データ取得失敗 ({e})")
    # 戻り値
    return all_horses_data

def scrape_race_data(driver, race_id, date):

    try:
        # 競馬場
        racecourse = driver.find_element(By.CSS_SELECTOR, SELECTOR_RACECOURSE).text
        # R
        race_number = driver.find_element(By.CSS_SELECTOR, SELECTOR_RACE_NUMBER).text
        # 頭数
        num_horses = driver.find_element(By.CSS_SELECTOR, SELECTOR_NUM_HORSES).text
        num_horses = int(num_horses.replace("頭", "").strip())
        # レース情報（発走時刻、グラウンド、距離、向き）
        race_info = driver.find_element(By.XPATH, XPATH_RACE_INFO).text
        race_info = race_info.strip()
        # "/"で分割
        parts = [p.strip() for p in race_info.split("/")]            
        # 発走時刻を抽出
        race_time = parts[0].replace("発走", "").strip()
        ground_info = parts[1]
        # グラウンドを抽出
        ground_match = re.search(r"(芝|ダ)", ground_info)
        ground = ground_match.group(1) if ground_match else "不明"
        # 距離を抽出
        distance_match = re.search(r"(\d+)m", ground_info)
        distance = int(distance_match.group(1)) if distance_match else -1
        # 向きを抽出
        direction_match = re.search(r"\((左|右)\)", ground_info)
        direction = direction_match.group(1) if direction_match else "不明"
        # 馬のデータを抽出
        if MODE == "shutuba":
            horse_list = scrape_horse_shutuba_data(driver, num_horses)
        elif MODE == "result":
            horse_list = scrape_horse_result_data(driver, num_horses)
        # 競馬場が帯広でない場合は予想データを取得する
        reliability = None
        opinion = None
        rank_1_corner = None
        rank_2_corner = None
        rank_3_corner = None
        rank_4_corner = None
        if MODE == "shutuba" and racecourse != "帯広(ば)":
            if is_xpath_present(driver, XPATH_CORNER_1):
                # 1人気信頼度
                if is_xpath_present(driver, XPATH_RELIABILITY):
                    reliability = driver.find_element(By.XPATH, XPATH_RELIABILITY).text
                # 見解
                if is_xpath_present(driver, XPATH_OPINION):
                    opinion = driver.find_element(By.XPATH, XPATH_OPINION).text
                # ポジション予想（1）
                corner = driver.find_element(By.XPATH, XPATH_CORNER_1).text
                dic_position_pred = get_position_pred(driver)
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                # ポジション予想（2）
                driver.find_element(By.XPATH, XPATH_CORNER_2).click()
                corner = driver.find_element(By.XPATH, XPATH_CORNER_2).text
                dic_position_pred = get_position_pred(driver)
                # ポジション予想を馬と紐づける
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                # ポジション予想（3）
                driver.find_element(By.XPATH, XPATH_CORNER_3).click()
                corner = driver.find_element(By.XPATH, XPATH_CORNER_3).text
                dic_position_pred = get_position_pred(driver)
                # ポジション予想を馬と紐づける
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                # 「騎手傾向を考慮」をチェック
                driver.find_element(By.XPATH, "//*[@id='dev_check_01_03']").click()
                # ポジション予想（1）
                driver.find_element(By.XPATH, XPATH_CORNER_1).click()
                corner = driver.find_element(By.XPATH, XPATH_CORNER_1).text
                dic_position_pred = get_position_pred(driver)
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                # ポジション予想（2）
                driver.find_element(By.XPATH, XPATH_CORNER_2).click()
                corner = driver.find_element(By.XPATH, XPATH_CORNER_2).text
                dic_position_pred = get_position_pred(driver)
                # ポジション予想を馬と紐づける
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                # ポジション予想（3）
                driver.find_element(By.XPATH, XPATH_CORNER_3).click()
                corner = driver.find_element(By.XPATH, XPATH_CORNER_3).text
                dic_position_pred = get_position_pred(driver)
                # ポジション予想を馬と紐づける
                match corner:
                    case "スタート後":
                        for horse in horse_list:
                            horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "2コーナー":
                        for horse in horse_list:
                            horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "3コーナー":
                        for horse in horse_list:
                            horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                    case "4コーナー":
                        for horse in horse_list:
                            horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                            horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]                
            # 「旧版の展開予想に切り替える」をクリックして脚質予想を取得
            driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[14]/div/div/label/span").click()
            list_lead = []
            list_stalker = []
            list_chaser = []
            list_closer = []
            # 逃げ
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]"):
                    list_lead.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 先行
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]"):
                    list_stalker.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 差し
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]"):
                    list_chaser.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 追込
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]"):
                    list_closer.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 脚質予想を馬と紐づける
            for horse in horse_list:
                if horse.horse_number in list_lead:
                    horse.style_pred = STYLE_MAP.get(1)
                elif horse.horse_number in list_stalker:
                    horse.style_pred = STYLE_MAP.get(2)
                elif horse.horse_number in list_chaser:
                    horse.style_pred = STYLE_MAP.get(3)
                elif horse.horse_number in list_closer:
                    horse.style_pred = STYLE_MAP.get(4)
        elif MODE == "result" and racecourse != "帯広(ば)":
            if is_xpath_present(driver, XPATH_RANK_1_CORNER):
                rank_1_corner = driver.find_element(By.XPATH, XPATH_RANK_1_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_2_CORNER):
                rank_2_corner = driver.find_element(By.XPATH, XPATH_RANK_2_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_3_CORNER):
                rank_3_corner = driver.find_element(By.XPATH, XPATH_RANK_3_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_4_CORNER):
                rank_4_corner = driver.find_element(By.XPATH, XPATH_RANK_4_CORNER).text
        # レースデータに格納
        race_data = Race(
            race_id = race_id,
            # race_name
            race_date = date,
            race_time = race_time,
            num_horses = num_horses,
            race_number = race_number,
            # weather_name
            # track_condition_name,
            racecourse = racecourse,
            ground = ground,
            distance = distance,
            direction = direction,
            reliability = reliability,
            opinion = opinion,
            rank_1_corner = rank_1_corner,
            rank_2_corner = rank_2_corner,
            rank_3_corner = rank_3_corner,
            rank_4_corner = rank_4_corner,
            horses = horse_list
        )
        # 出力確認
        print(f"\n📄 年月日: {race_data.race_date} | 競馬場: {race_data.racecourse} | R: {race_data.race_number}")
        for horse in race_data.horses:
            print(f"  馬番: {horse.horse_number} | 馬名: {horse.horse_name}")
    
    except Exception as e:
        print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
        return None
    # 戻り値
    return race_data

def create_html_body(df_buy, is_success):
    # デザイン設定
    color_success = "#28a745" # 緑
    color_fail = "#dc3545"    # 赤
    color_none = "#6c757d"    # グレー
    
    if not df_buy.empty:
        status_color = color_success if is_success else color_fail
        status_text = "購入成功" if is_success else "購入失敗"
        
        # DataFrameをHTMLの表に変換
        # classesやborderなどを指定してデザインの土台を作る
        table_html = df_buy[['race_id', 'horse_number', 'popularity', 'expected_value']].to_html(
            index=False, 
            border=1, 
            justify='center'
        )
        # 表のデザインをさらに細かく調整（置換でスタイルを注入）
        table_html = table_html.replace(
            '<table border="1" class="dataframe">', 
            '<table style="border-collapse: collapse; width: 100%; text-align: center; font-family: sans-serif;">'
        ).replace(
            '<th>', '<th style="background-color: #f2f2f2; padding: 8px; border: 1px solid #ddd;">'
        ).replace(
            '<td>', '<td style="padding: 8px; border: 1px solid #ddd;">'
        )
        
        content = f"""
        <div style="font-family: sans-serif; max-width: 600px; margin: auto; border: 1px solid #ddd; padding: 20px;">
            <h2 style="color: {status_color}; border-bottom: 2px solid {status_color}; padding-bottom: 10px;">
                {status_text}
            </h2>
            <p>購入結果</p>
            {table_html}
        </div>
        """
    else:
        content = f"""
        <div style="font-family: sans-serif; max-width: 600px; margin: auto; border: 1px solid #ddd; padding: 20px;">
            <h2 style="color: {color_none}; border-bottom: 2px solid {color_none}; padding-bottom: 10px;">
                購入対象なし
            </h2>
            <p>今回のレースに購入条件を満たす馬はいませんでした</p>
        </div>
        """
    
    footer = """
    <div style="font-size: 12px; color: #888; margin-top: 20px; text-align: center;">
        <hr>
        <p>このメールは競馬自動ツールから自動送信されています</p>
    </div>
    """
    return content + footer

def send_gmail(subject, body, to_address):
    # --- 設定項目 ---
    from_address = GMAIL_MAP.get('from')
    app_password = GMAIL_MAP.get('app_password')

    # --- メッセージの作成 ---
    msg = MIMEMultipart()
    msg['From'] = from_address
    msg['To'] = to_address
    msg['Subject'] = subject

    # --- 本文の追加 ---
    # もし body が <html> で始まっている場合は 'html' として、
    # そうでない場合は 'plain' として送る切り替えを入れる
    if body.strip().startswith('<'):
        msg.attach(MIMEText(body, 'html'))
    else:
        msg.attach(MIMEText(body, 'plain'))

    try:
        # --- SMTPサーバーへの接続と送信 ---
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        
        server.login(from_address, app_password)
        server.send_message(msg)
        server.quit()
        
        print("メールを送信しました")
        return True
    except Exception as e:
        print(f"送信失敗: {e}")
        return False

In [8]:
# =================
# 予想データ読み込み
# =================
### predデータ読み込み ###
print("predデータ読み込み：開始")
df_pred_raw = pd.read_csv("C:\\keiba\\tool\\pred\\pred.csv", header=0, encoding='cp932', dtype={'race_id': str})
print("predデータ読み込み：終了")

predデータ読み込み：開始
predデータ読み込み：終了


In [10]:
df_pred = df_pred_raw.copy()
# 購入対象のレースを取得
df_race = df_pred[['race_id', 'race_time', 'racecourse', 'race_number']].drop_duplicates(subset=['race_id'])
# 時刻の昇順にソート
df_race = df_race.sort_values('race_time').reset_index(drop=True)
# target_times リストを作成
target_times = df_race['race_time'].tolist()
print(f"取得した時刻リスト: {target_times}")

取得した時刻リスト: ['14:50', '15:25', '16:00', '16:35', '17:45', '18:15', '18:45', '19:20', '19:50']


In [ ]:
######################
# 最新データ取得～購入
######################

# 時刻リストをループ
for t_str in sorted(target_times):
    # 今日の日付と組み合わせて datetime オブジェクトにする
    now = datetime.now()
    target_dt = datetime.strptime(f"{now.strftime('%Y-%m-%d')} {t_str}", "%Y-%m-%d %H:%M")    
    # 決めた時間分マイナスする（実行予定時刻）
    exec_dt = target_dt - timedelta(minutes=SPAT4_MAP.get('PRE_MINUTES'))
    # すでにその時刻を過ぎている場合はスキップ
    if exec_dt < now:
        print(f"スキップ: {t_str} (実行予定時刻 {exec_dt.strftime('%H:%M')} を過ぎています)")
        continue
    # 実行時刻まで待機
    wait_seconds = (exec_dt - now).total_seconds()
    print(f"待機中... 次の実行は {exec_dt.strftime('%H:%M')} です（あと {int(wait_seconds)} 秒）")
    # 実行時刻まで待機する    
    time.sleep(wait_seconds)

    # ここでさっきの Selenium 処理などを呼び出す
    current_race = df_race[df_race['race_time'] == t_str].iloc[0].to_dict()
    race_id = current_race['race_id']
    racecourse = current_race['racecourse']  
    race_number = current_race['race_number']    
    
    print(f"時刻: {t_str} のレースを処理します")
    print(f"ID: {race_id}")
    print(f"場所: {racecourse}")
    print(f"番号: {race_number}")
    
    # ===============
    # 人気・オッズ更新
    # ===============
    
    # スクレイピング準備
    service = Service(PATH_CHROME_DRIVER)
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    # ログイン
    login_netkeiba(driver)
    # データ取得
    race_url = f'https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}'
    driver.get(race_url)
    time.sleep(1)
    race_data = scrape_race_data(driver, race_id, None)
    # スクレイピング終了
    driver.quit()
    # 予想データ更新
    update_rows = []
    for horse in race_data.horses:
        update_rows.append({
            'race_id': race_data.race_id,
            'horse_number': horse.horse_number,
            'new_popularity': horse.popularity,
            'new_odds': horse.odds
        })
    df_update = pd.DataFrame(update_rows)
    df_update['horse_number'] = df_update['horse_number'].astype(int)
    df_pred = pd.merge(
        df_pred, 
        df_update, 
        on=['race_id', 'horse_number'], 
        how='left'
    )
    df_pred['popularity'] = df_pred['new_popularity'].fillna(df_pred['popularity'])
    df_pred['odds'] = df_pred['new_odds'].fillna(df_pred['odds'])
    df_pred = df_pred.drop(columns=['new_popularity', 'new_odds'])

    # ==========
    # 期待値計算
    # ==========
    df_pred['odds'] = pd.to_numeric(df_pred['odds'], errors='coerce')
    df_pred['win_rate'] = pd.to_numeric(df_pred['win_rate'], errors='coerce')
    df_pred['expected_value'] = df_pred['win_rate'] * df_pred['odds']

    # ==========
    # 購入判定
    # ==========
    
    df_pred['map_key'] = (
        'th_num' + df_pred['num_horses'].astype(str) + 
        '_pop' + df_pred['popularity'].astype(str)
    )
    df_pred['threshold'] = df_pred['map_key'].map(PURCHASE_MAP)
    df_buy = df_pred[
        (df_pred['race_id'] == race_id) &
        (df_pred['threshold'].notna()) & 
        (df_pred['expected_value'] >= df_pred['threshold'])
    ].copy()
    df_buy = df_buy.drop(columns=['map_key', 'threshold'])

    ###############
    # 購入（spat4）
    ###############
    
    if not df_buy.empty:
        print("購入：開始")
        try:
            # spat4で選択できるように帯広(ば) → 帯広に修正
            if racecourse == '帯広（ば）':
                racecourse_to_select = racecourse.lstrip("(ば)")
            else:
                racecourse_to_select = racecourse
            # spat4でスクレイピング
            driver = webdriver.Chrome(service=service, options=options)
            # ログイン
            login_spat4(driver)
            # オッズ投票画面に遷移
            if is_xpath_present(driver, "/html/body/div[2]/div[1]/table/tbody/tr[2]/td[5]/a/span/div"):
                driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/table/tbody/tr[2]/td[5]/a/span/div").click()
            elif is_xpath_present(driver, "/html/body/div[2]/div[1]/table[1]/tbody/tr[2]/td[5]/a/span/div"):
                driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/table[1]/tbody/tr[2]/td[5]/a/span/div").click()
            # 競馬場を選択
            driver.switch_to.frame("LEFT")
            dropdown_element = driver.find_element(By.NAME, "PLACE")
            select_obj = Select(dropdown_element)
            select_obj.select_by_visible_text(racecourse_to_select)
            # Rを選択
            dropdown_element = driver.find_element(By.NAME, "RACE")
            select_obj = Select(dropdown_element)
            select_obj.select_by_visible_text(race_number)
            # 単勝オッズを選択
            for row in df_buy.itertuples():
                horse_number = int(row.horse_number)
                if is_xpath_present(driver, f"/html/body/form[2]/table/tbody/tr[{str(horse_number + 2)}]/td[5]"):
                    driver.find_element(By.XPATH, f"/html/body/form[2]/table/tbody/tr[{str(horse_number + 2)}]/td[4]").click()
                else:
                    driver.find_element(By.XPATH, f"/html/body/form[2]/table/tbody/tr[{str(horse_number + 2)}]/td[3]").click()
            # 金額入力
            driver.switch_to.default_content()
            driver.switch_to.frame("RIGHT")
            i = 1
            for row in df_buy.itertuples():       
                driver.find_element(By.XPATH, f"//*[@id='TEXTMONEY_{str(i)}']").send_keys("1")
                i = i + 1
            # 「投票内容確認へ」をクリック
            driver.find_element(By.XPATH, "/html/body/form/center/table[3]/tbody/tr/td/font/nobr/input[1]").click()
            # 「投票内容確認へ移ります。よろしいですか？」を次に進むためにENTER
            wait = WebDriverWait(driver, 5)
            wait.until(EC.alert_is_present())    
            alert = driver.switch_to.alert
            alert.accept()
            driver.switch_to.window(driver.window_handles[-1])
            driver.switch_to.default_content()
            # 暗証番号入力
            time.sleep(1)
            driver.find_element(By.NAME, "MEMBERPASSR").send_keys(SPAT4_MAP.get('暗証番号'))
            # 投票金額入力
            driver.find_element(By.XPATH, "//*[@id='TOTALMONEYR']").send_keys(str(len(df_buy) * 100))
            # 投票する押下
            if SPAT4_MAP.get('MODE') == 'prod':
                driver.find_element(By.XPATH, "/html/body/div/center/form/table/tbody/tr/td/input").click()
                # 「投票を実行します。よろしいですか？」を次に進むためにENTER
                wait = WebDriverWait(driver, 5)
                wait.until(EC.alert_is_present())
                alert = driver.switch_to.alert
                alert.accept()
                time.sleep(5)
            # 閉じる
            driver.quit()
            print("購入：成功")
            is_success = True
        except Exception as e:
            print("購入：失敗")
            is_success = False
        finally:
            print("購入：終了")
    else:
        print("購入：なし")

    # ==========
    # メール送信
    # ==========
        
    # --- メイン処理部分 ---
    to_address = GMAIL_MAP.get('to')
    
    if not df_buy.empty and is_success:
        subject = "【競馬ツール】購入完了【〇】"
    elif not df_buy.empty and not is_success:
        subject = "【競馬ツール】購入失敗【×】"
    else:
        subject = "【競馬ツール】対象なし【－】"
    
    body_html = create_html_body(df_buy, is_success)
    
    # 送信（send_gmailの中でMIMETextの第二引数を'html'にする必要があります）
    send_gmail(subject, body_html, to_address)
    
    print("処理が完了しました。次のスケジュールへ移行します。")

print("すべてのスケジュールを終了")

スキップ: 14:50 (実行予定時刻 14:13 を過ぎています)
スキップ: 15:25 (実行予定時刻 14:48 を過ぎています)
スキップ: 16:00 (実行予定時刻 15:23 を過ぎています)
スキップ: 16:35 (実行予定時刻 15:58 を過ぎています)
スキップ: 17:45 (実行予定時刻 17:08 を過ぎています)
スキップ: 18:15 (実行予定時刻 17:38 を過ぎています)
スキップ: 18:45 (実行予定時刻 18:08 を過ぎています)
待機中... 次の実行は 18:43 です（あと 45 秒）
時刻: 19:20 のレースを処理します
ID: 202655012409
場所: 佐賀
番号: 9R

📄 年月日: None | 競馬場: 佐賀 | R: 9R
  馬番: 1 | 馬名: ナッシングオンユー
  馬番: 2 | 馬名: トリオ
  馬番: 3 | 馬名: ラピダッシュ
  馬番: 4 | 馬名: ゴルトツィーゲル
  馬番: 5 | 馬名: サンネリネ
  馬番: 6 | 馬名: サザンフィールド
  馬番: 7 | 馬名: エミネンスローズ
  馬番: 8 | 馬名: サブアトミック
  馬番: 9 | 馬名: シントーキズナ
  馬番: 10 | 馬名: ツィードリヴァ
  馬番: 11 | 馬名: カンタベリーハヤテ
  馬番: 12 | 馬名: パワポケチャンプ
購入：開始
購入：成功
購入：終了
メールを送信しました
処理が完了しました。次のスケジュールへ移行します。
待機中... 次の実行は 19:13 です（あと 1752 秒）


In [ ]:
# 購入履歴ファイル作成
# 時刻の5分前になったら処理開始。レースごとにループさせる（これは最後。まずは1レースで実現する）

In [ ]:
# ===============
# 出力
# ===============

df_pred.to_csv("C:\\keiba\\tool\\pred\\pred.csv", index=False, encoding="cp932")
# df_purchase_records.to_csv("C:\\keiba\\tool\\pred\\purchase_records.csv", index=False, encoding="cp932")